In [1]:
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, Markdown
from langchain_openai import AzureChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")

In [2]:
def init_llm(model=OPENAI_GPT35_DEPLOYMENT_NAME,
             deployment_name=OPENAI_GPT35_DEPLOYMENT_NAME,
             openai_api_version="2024-02-15-preview",
             temperature=0,
             max_tokens=400
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                            model=model,
                            openai_api_version=openai_api_version,
                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                            temperature=temperature,
                            max_tokens=max_tokens
                            )
    return llm

llm = init_llm()

##### Let's do some math

In [8]:
# openAI gets this wrong!!
prompt = f""" Determine if the student's solution delimited by the triple backticks is correct or not.
Question: When I was 2 years old my sister was twice my age. I'm now 40 years old how old is my sister now? 
Student's answer: ```The sister is now 80 years old.``` 
If student is correct, then write 'Correct', otherwise write 'Incorrect'.
"""

sum = llm.invoke(prompt)
display(HTML(sum.content))

##### Ask the model to solve it and then to compare both solutions and conclude which is correct.
 Define the task as a list of sub-tasks and ask the model to perform them in a specific order (splitting the task into sub-tasks technique).   
 Then ask the model to compare its own solution with the solution provided by the model and conclude which one is correct.
 Ask the model to share its reasoning for the conclusion.

In [9]:
Question = f"""When I was 2 years old my sister was twice of my age . I'm now 40 years old, how old is my sister now?"""
Student_Solution = f""" The sister is now 80 years old."""

prompt = f""" Determine if the Student's Solution for the Question is correct or not.
To solve the problem do the following:
1 - First, work out your OWN solution to the problem. Evaluate your final result to make sure it is correct and adheres to the question's conditions. 
    Reason about every step of your solution and make sure it is correct. 
2 - Second, compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Student's Solution: ```{Student_Solution}```
Question: ```{Question}```


Use the following output format:
Actual solution steps: <your own solution steps>
Student's solution: <student's solution>
Student's solution is correct: <true/false>

"""
sum = llm.invoke(prompt)
display(Markdown(sum.content))

Actual solution steps: 
1 - When I was 2 years old, my sister was twice my age, so she was 4 years old.
2 - The age difference between me and my sister is always 2 years.
3 - I am now 40 years old, so my sister is 40 - 2 = 38 years old.

Student's solution: The sister is now 80 years old.

Student's solution is correct: False